# 弱凸関数での確率的最適化

参考：
* [STOCHASTIC MODEL-BASED MINIMIZATION OF WEAKLY CONVEX FUNCTIONS](https://arxiv.org/abs/1803.06523)
  * Introの部分にStochastic Optimizationの歴史が書いてあるので，読んだほうがいい．
  * 6ページ後半から書いてあるが，簡単なケースでもこの論文は貢献があるらしい．
    * $f(\cdot, \xi)$が平滑で$r$が指示関数の場合
    * $f$が非平滑で$r=0$の場合 
  * 8ページにWeakly convexの初出がある？
* [STOCHASTIC METHODS FOR COMPOSITE ANDWEAKLY CONVEX OPTIMIZATION PROBLEM](https://epubs.siam.org/doi/abs/10.1137/17M1135086?casa_token=pBpgYK-h3cEAAAAA:FVeg27uwY1Gy23Mj1IRNTbFeBmlLS5eVMwBEehGaoGKKOkbcxiia1ggvNbOhvzqqQsXEkjcW5w)これも読むといいかも

次の問題を考えましょう：

$$
\min _{x \in \mathbb{R}^d} \varphi(x)=f(x)+r(x) \quad \text { where } \quad f(x)=\mathbb{E}_{\xi \sim P}[f(x, \xi)] \text {. }
$$

* ここで，$f(x, \xi)$は$x$によってパラメータ付けされたときの意思決定が，データ$\xi$で生じる損失を表してます．Domain randomizationとか考えるとイメージしやすいですね．
* $r: \mathbb{R}^d \rightarrow \mathbb{R} \cup\{+\infty\}$は$x$の制約とかを表現します．
* $f$は$\rho$-弱凸関数とします．$r$は閉じた凸関数です．

$f(\cdot, \xi)$が平滑＆凸＆$r=0$のとき，一番シンプルな確率的凸最適化問題になります．


表記：
* 関数 $\varphi: \mathbb{R}^d \rightarrow \mathbb{R} \cup\{\infty\}$ の劣勾配：$\varphi(y) \geq \varphi(x)+\langle v, y-x\rangle+o(\|y-x\|) \quad$ as $y \rightarrow x$
* モーロー包：$\varphi_\lambda(x):=\min _y\left\{\varphi(y)+\frac{1}{2 \lambda}\|y-x\|^2\right\}$
* 近接射影：$\operatorname{prox}_{\lambda \varphi}(x):=\underset{y}{\operatorname{argmin}}\left\{\varphi(y)+\frac{1}{2 \lambda}\|y-x\|^2\right\}$

$\hat{x}:=\operatorname{prox}_{\lambda \varphi}(x)$について，次の３つが成立する：
1. モーロー包の勾配は近接射影の停留性を表す：$\|\hat{x}-x\| \quad=\lambda\left\|\nabla \varphi_\lambda(x)\right\|$
2. 近接射影の法が値が小さくなる：$\varphi(\hat{x}) \quad \leq \varphi(x)$
3. 劣勾配はモーロー包の勾配で抑えられる：$\operatorname{dist}(0 ; \partial \varphi(\hat{x})) \quad \leq\left\|\nabla \varphi_\lambda(x)\right\|$

また，モーロー包の勾配は近接勾配法のGradient mappingとだいたい同じ量になります（Gradient mappingの定義は論文参照）：
$$
\frac{1}{4}\left\|\nabla \varphi_{1 / 2 \rho}(x)\right\| \leq\left\|\mathcal{G}_{1 / \rho}(x)\right\| \leq \frac{3}{2}\left(1+\frac{1}{\sqrt{2}}\right)\left\|\nabla \varphi_{1 / 2 \rho}(x)\right\| \quad \forall x \in \mathbb{R}^d
$$

仮定：
* $\xi_1, \xi_2, \ldots \sim P$からIIDサンプル可能
* 劣勾配の推定機：$\operatorname{dom} r$を含んだ開集合$U$が存在し，$\mathbb{E}_{\xi}[G(x, \xi)] \in \partial f(x)$ for all $x \in U$を満たす$G: U \times \Omega \to \mathbb{R}^d$が存在する
* 劣勾配のノルム：任意の$x \in \operatorname{dom} r$に対して，$\mathbb{E}_{\xi}\left[\|G(x, \xi)\|^2\right] \leq L^2$

以降，$\mathbb{E}_t[\cdot]$は$\xi_0, \dots, \xi_{t-1}$で条件付けられたときの期待値を表す．

今回は，この仮定のもとで，次の確率的近接劣勾配法を考えます．

* 入力：$x_0 \in \operatorname{dom} r$, $\{\alpha_t\}_{t\geq 0} \subset \mathbb{R}_+$, イテレーションカウント$T$
* $t=0\dots T$について，
  * $\xi_t \sim P$をサンプリング
  * $x_{t+1}=\operatorname{prox}_{\alpha_t r}(x_t-\alpha_t G(x_t, \xi_t))$
* 次の$t^*$をサンプル：$t^* \in\{0, \ldots, T\}$ according to $\mathbb{P}\left(t^*=t\right)=\frac{\alpha_t}{\sum_{i=0}^T \alpha_i}$
* $x_{t^*}$を出力

## 射影劣勾配法のとき

射影作用素は近接作用素の一種なので，$\operatorname{proj}_\mathcal{X}$なる近接作用素を使って，次の問題を解くことを考えましょう：
$$
\min _{x \in \mathcal{X}} f(x)
$$

この収束について解析しましょう．つまり，$f$が$\rho$-弱凸＆$r$が指示関数のときですね．

**収束解析**

任意の$\bar{\rho} > $\rho$を考えましょう．
各更新ステップ$t$について，
$$
v_t:=\mathbb{E}_t\left[G\left(x_t, \xi\right)\right] \in \partial f\left(x_t\right) \text { and set } \hat{x}_t:=\operatorname{prox}_{\varphi / \bar{\rho}}\left(x_t\right) .
$$
とします．
つまり，十分な凸性を持った関数で近接射影してるわけですね．

まず，モーロー包の定義と$t$までの条件付から，明らかに，
$$
\begin{aligned}
\mathbb{E}_t\left[\varphi_{1 / \bar{\rho}}\left(x_{t+1}\right)\right] & \leq \mathbb{E}_t\left[f\left(\hat{x}_t\right)+\frac{\bar{\rho}}{2}\left\|\hat{x}_t-x_{t+1}\right\|^2\right] \\
& =f\left(\hat{x}_t\right)+\frac{\bar{\rho}}{2} \mathbb{E}_t\left[\left\|\operatorname{proj}_{\mathcal{X}}\left(x_t-\alpha_t G\left(x_t, \xi_t\right)\right)-\operatorname{proj}_{\mathcal{X}}\left(\hat{x}_t\right)\right\|^2\right] \\
\end{aligned}
$$
です．Projectionの性質から，
$$
\begin{aligned}
& \leq f\left(\hat{x}_t\right)+\frac{\bar{\rho}}{2} \mathbb{E}_t\left[\left\|\left(x_t-\hat{x}_t\right)-\alpha_t G\left(x_t, \xi_t\right)\right\|^2\right] \\
& \leq f\left(\hat{x}_t\right)+\frac{\bar{\rho}}{2}\left\|x_t-\hat{x}_t\right\|^2+\bar{\rho} \alpha_t \mathbb{E}_t\left[\left\langle\hat{x}_t-x_t, G\left(x_t, \xi_t\right)\right\rangle\right]+\frac{\alpha_t^2 \bar{\rho} L^2}{2} \\
& \leq \varphi_{1 / \bar{\rho}}\left(x_t\right)+\bar{\rho} \alpha_t\left\langle\hat{x}_t-x_t, v_t\right\rangle+\frac{\alpha_t^2 \bar{\rho} L^2}{2} \\
\end{aligned}
$$
が成立します．最後に，劣勾配の性質を使って，
$$
\begin{aligned}
& \leq \varphi_{1 / \bar{\rho}}\left(x_t\right)+\bar{\rho} \alpha_t\left(f\left(\hat{x}_t\right)-f\left(x_t\right)+\frac{\rho}{2}\left\|x_t-\hat{x}_t\right\|^2\right)+\frac{\alpha_t^2 \bar{\rho} L^2}{2},
\end{aligned}
$$
が成り立ちます．

次に，$x \mapsto f(x)+\frac{\bar{\rho}}{2}\left\|x-x_t\right\|^2$は強凸関数なので，
$$
\begin{aligned}
f\left(x_t\right)-f\left(\hat{x}_t\right)-\frac{\rho}{2}\left\|x_t-\hat{x}_t\right\|^2= & \left(f\left(x_t\right)+\frac{\bar{\rho}}{2}\left\|x_t-x_t\right\|^2\right)-\left(f\left(\hat{x}_t\right)+\frac{\bar{\rho}}{2}\left\|x_t-\hat{x}_t\right\|^2\right) \\
& +\frac{\bar{\rho}-\rho}{2}\left\|x_t-\hat{x}_t\right\|^2 \\
\geq & (\bar{\rho}-\rho)\left\|x_t-\hat{x}_t\right\|^2=\frac{\bar{\rho}-\rho}{\bar{\rho}^2}\left\|\nabla \varphi_{1 / \bar{\rho}}\left(x_t\right)\right\|^2,
\end{aligned}
$$
が成立します．よって，
$$
\mathbb{E}_t\left[\varphi_{1 / \bar{\rho}}\left(x_{t+1}\right)\right] \leq \varphi_{1 / \bar{\rho}}\left(x_t\right)-\frac{\alpha_t(\bar{\rho}-\rho)}{\bar{\rho}}\left\|\nabla \varphi_{1 / \bar{\rho}}\left(x_t\right)\right\|^2+\frac{\alpha_t^2 \bar{\rho} L^2}{2}
$$
が成立します．後は期待値を全体で取る（tower ruleを使う）と，

$$
\mathbb{E}\left[\varphi_{1 / \bar{\rho}}\left(x_{t+1}\right)\right] \leq \mathbb{E}\left[\varphi_{1 / \bar{\rho}}\left(x_t\right)\right]-\frac{\alpha_t(\bar{\rho}-\rho)}{\bar{\rho}} \mathbb{E}\left[\left\|\nabla \varphi_{1 / \bar{\rho}}\left(x_t\right)\right\|^2\right]+\frac{\alpha_t^2 \bar{\rho} L^2}{2}
$$
が成立します．

---

**ポイント** これはDescent lemma（[上界補題](OPT_Grad_convergence_basic.ipynb)）ってやつをモーロー包に適用したものですね．変形すると，

$$
\mathbb{E}\left[\varphi_{1 / \bar{\rho}}\left(x_t\right)\right]
- \mathbb{E}\left[\varphi_{1 / \bar{\rho}}\left(x_{t+1}\right)\right] 
\geq 
\frac{\alpha_t(\bar{\rho}-\rho)}{\bar{\rho}} \mathbb{E}\left[\left\|\nabla \varphi_{1 / \bar{\rho}}\left(x_t\right)\right\|^2\right]-\frac{\alpha_t^2 \bar{\rho} L^2}{2}
$$
であり，更新ごとのモーロー包の減少量を表しています．

---

この繰り返しを変形すると，
$$
\mathbb{E}\left[\varphi_{1 / \bar{\rho}}\left(x_{T+1}\right)\right] \leq \varphi_{1 / \bar{\rho}}\left(x_0\right)+\frac{\bar{\rho} L^2}{2} \sum_{t=0}^T \alpha_t^2-\frac{\bar{\rho}-\rho}{\bar{\rho}} \cdot \sum_{t=0}^T \alpha_t \mathbb{E}\left[\left\|\nabla \varphi_{1 / \bar{\rho}}\left(x_t\right)\right\|^2\right]
$$

が出てきます．後は変形して，
$$
\mathbb{E}\left[\left\|\nabla \varphi_{1 / \bar{\rho}}\left(x_{t^*}\right)\right\|^2\right] \leq \frac{\bar{\rho}}{\bar{\rho}-\rho} \cdot \frac{\left(\varphi_{1 / \bar{\rho}}\left(x_0\right)-\min \varphi\right)+\frac{\bar{\rho} L^2}{2} \sum_{t=0}^T \alpha_t^2}{\sum_{t=0}^T \alpha_t}
$$
が得られます．